# Data Drift Iris dataset

In [1]:
import pandas as pd

from sklearn import datasets

from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab

from evidently.model_profile import Profile
from evidently.profile_sections import DataDriftProfileSection

## Iris Data

In [2]:
iris = datasets.load_iris()

In [3]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

## Data Drift Dashboard

In [4]:
iris_data_drift_dashboard = Dashboard(tabs=[DataDriftTab])
iris_data_drift_dashboard.calculate(iris_frame, iris_frame, column_mapping = None)

In [5]:
iris_data_drift_dashboard.show()

In [6]:
#iris_data_drift_dashboard.save('iris_data_drift.html')

## Data Drift Profile

In [7]:
iris_data_drift_profile = Profile(sections=[DataDriftProfileSection])
iris_data_drift_profile.calculate(iris_frame, iris_frame, column_mapping = None)

In [8]:
iris_data_drift_profile.json()

{"data_drift": {"name": "data_drift", "datetime": "2021-07-15 20:10:10.126000", "data": {"utility_columns": {"date": null, "id": null, "target": null, "prediction": null}, "cat_feature_names": [], "num_feature_names": ["petal length (cm)", "sepal length (cm)", "petal width (cm)", "sepal width (cm)"], "metrics": {"petal length (cm)": {"current_small_hist": [[0.4180790960451977, 0.14689265536723162, 0.0, 0.033898305084745776, 0.09039548022598871, 0.2937853107344629, 0.32768361581920913, 0.20338983050847462, 0.12429378531073448, 0.05649717514124295], [1.0, 1.59, 2.18, 2.7700000000000005, 3.3600000000000003, 3.95, 4.540000000000001, 5.130000000000001, 5.720000000000001, 6.3100000000000005, 6.9]], "ref_small_hist": [[0.4180790960451977, 0.14689265536723162, 0.0, 0.033898305084745776, 0.09039548022598871, 0.2937853107344629, 0.32768361581920913, 0.20338983050847462, 0.12429378531073448, 0.05649717514124295], [1.0, 1.59, 2.18, 2.7700000000000005, 3.3600000000000003, 3.95, 4.540000000000001, 5